In [23]:
def get_current_dir():
    import os
    from pathlib import Path
    try:
        return Path(__file__).parent.absolute()
    except NameError:
        return Path(os.getcwd())

path = get_current_dir().parent / "data" / "results.csv"

import pandas as pd
df = pd.read_csv(path)

# from table in paper
for alpha_ent in [0.30, 0.40]:
    for alpha_word in [0.2, 0.3, 0.4, 0.5]:
        for k in [20, 50, 100, 200]:

            # common values
            for n_topics in [10, 20, 50, 100]:
                for n_neighbours in [5, 10, 20, 500]: # 500 is just default in code

                    # assert exists
                    mask = (df.n_topics == n_topics) & (df.n_neighbours == n_neighbours) & (df.alpha_ent == alpha_ent) & (df.k == k) & (df.alpha_word == alpha_word)
                    assert not df[mask].empty

In [ ]:
# from table in paper
for alpha_ent in [0.30, 0.40]:
    for alpha_word in [0.2, 0.3, 0.4, 0.5]:
        for k in [20, 50, 100, 200]:

            

            # common values
            for n_topics in [10, 20, 50, 100]:
                for n_neighbours in [5, 10, 20, 500]: # 500 is just default in code

                    # assert exists
                    mask = (df.n_topics == n_topics) & (df.n_neighbours == n_neighbours) & (df.alpha_ent == alpha_ent) & (df.k == k) & (df.alpha_word == alpha_word)
                    assert not df[mask].empty